In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import shap
import matplotlib.pyplot as plt
import xgboost as xgb

df = pd.read_csv('synthesized_data_zakat.csv')

/Users/muhdsafrul/Desktop/Dev/Repositories/MuamalatGo/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 47 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Kategori Asnaf                                15000 non-null  object 
 1   Dokumen Lampiran Utama                        15000 non-null  object 
 2   Kategori pemohon                              15000 non-null  object 
 3   Sebab Memohon Bantuan                         15000 non-null  object 
 4   No. K/P (baru)/Polis/Tentera/No. Pasport      15000 non-null  object 
 5   Nama Pemohon/Institusi                        15000 non-null  object 
 6   Alamat                                        15000 non-null  object 
 7   Daerah                                        15000 non-null  object 
 8   Poskod                                        15000 non-null  int64  
 9   Bandar                                        15000 non-null 

In [3]:
# Convert the 'Tarikh' column to datetime
df["Tarikh"] = pd.to_datetime(df["Tarikh"])

# Find the newest date in the 'Tarikh' column
reference_date = df["Tarikh"].max()

# Calculate the number of days since the reference date
df["Days Since"] = (reference_date - df["Tarikh"]).dt.days

In [4]:
df["Days Since"].value_counts()

Days Since
33    201
65    196
8     193
85    193
82    191
     ... 
9     144
49    142
7     141
73    140
59    131
Name: count, Length: 91, dtype: int64

In [5]:
# Drop the 'Tarikh' column
df = df.drop(columns=["Tarikh"])

warganegara

In [6]:
df["Poligami"] = df["Poligami"].fillna(0)

In [7]:
df["Poligami"].value_counts()

Poligami
0.0    14545
3.0      163
2.0      148
4.0      144
Name: count, dtype: int64

In [8]:
# Convert the 'Tarikh Lahir' column to datetime
df["Tarikh Lahir"] = pd.to_datetime(df["Tarikh Lahir"])
# Calculate age based on the reference date
df["Age"] = (reference_date - df["Tarikh Lahir"]).dt.days // 365  # Integer division for years
df = df.drop(columns=["Tarikh Lahir"])

df["Age"].value_counts()

Age
69    297
74    295
73    292
57    287
32    287
70    286
26    284
49    281
50    280
30    276
24    276
46    274
33    274
51    273
44    273
47    272
42    272
40    270
20    270
19    270
39    269
31    269
52    267
21    266
64    266
29    265
36    264
27    263
41    262
61    262
48    260
66    260
23    260
54    257
45    257
53    257
68    256
25    256
34    256
38    255
59    255
37    253
43    253
65    252
72    251
28    251
63    251
18    251
67    248
60    245
22    242
62    241
55    238
58    238
35    235
56    235
71    230
75     15
Name: count, dtype: int64

In [9]:
df = df.drop(columns=["Tarikh Masuk Islam"])

In [10]:
df["No. Telefon Rumah/Waris"] = df["No. Telefon Rumah/Waris"].apply(lambda x: "YA" if pd.notnull(x) else "TIADA")


In [11]:
df["No. Telefon Rumah/Waris"].value_counts()

No. Telefon Rumah/Waris
YA       10523
TIADA     4477
Name: count, dtype: int64

In [12]:
df["No. Telefon Bimbit"] = df["No. Telefon Bimbit"].apply(lambda x: "YA" if pd.notnull(x) else "TIADA")

In [13]:
df["No. Telefon Bimbit"].value_counts()

No. Telefon Bimbit
YA    15000
Name: count, dtype: int64

In [14]:
df["Tempoh Menetap di Selangor"].value_counts()

Tempoh Menetap di Selangor
1     405
35    403
14    403
20    400
12    400
2     397
19    397
4     395
18    394
8     393
34    393
13    389
33    389
11    387
40    385
26    381
25    380
6     378
21    374
16    374
9     373
10    372
7     371
38    371
30    370
24    367
17    365
23    363
39    361
29    361
15    361
5     360
3     358
37    358
36    357
31    351
32    349
27    348
22    335
28    332
Name: count, dtype: int64

In [15]:
df.to_csv("cleaned_data.csv", index=False)